# Testing Axonal Contamination

Axons propagate somatic signal within 5ms time - more or less instantly with respect to 50ms sampling timescale. So, observing axons from another population is effectively crosstalk between instantaneous signals.

Questions of interest:
* Do metrics perform well in strict sense?
* Do metrics perform well in loose sense?

**Model 1**: 2-node no link hypothesis
* 2 completely uncorrelated
* FC estimated at lags [1 to 5]
* Add symmetric cross-talk of a factor alpha, sweep over alpha in [0, 1]
* With and without kernel convolution

**Model 2**: 2-node one link hypothesis
* 2 regions interacting at 1 lag of [1 to 5] with fraction 0.5 and low noise
* FC estimated at lags [1 to 5]
* Add symmetric cross-talk of a factor alpha, sweep over alpha in [0, 1]
* With and without kernel convolution

Accuracy measures:
* Strict TP: Fraction tests where FC significant at right lag
* Strict FP: Fraction tests where FC significant at at least one wrong lag
* Loose TP: Fraction tests where highest FC at right lag and significant
* Loose FP: Fraction tests where highest FC at wrong lag and significant

**Model 2**: 3D effects
* 3 regions, FC between (1)->(2) at fixed lag
* Cross-talk between (1) and (3)
* Estimate only mTE
* Strict TP: Fraction tests where (1)->(2) significant
* Strict FP: Fraction tests where (3)->(2) significant

In [1]:
# Append base directory
import os,sys,inspect
rootname = "mesoscopic-functional-connectivity"
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)


%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/mesoscopic-functional-connectivity


In [2]:
from mesostat.metric.metric import MetricCalculator
mc = MetricCalculator(serial=True, verbose=False)

In [3]:
import code_python.lib.null.axonal_simulations as axsim

In [ ]:
axsim.crosstalk_test_two_node_negative(mc, ['crosscorr'], testConv=True, nCTDiscr=10,
                  nTest=50, maxLag=5, pTHR=0.01, noiseFrac=0.01, convDT=0.05, convTau=0.5)

Doing method crosscorr raw


In [ ]:
axsim.crosstalk_test_two_node_positive(mc, ['MultivariateTE'], testConv=True, nCTDiscr=10,
                  nTest=50, maxLag=5, pTHR=0.01, noiseFrac=0.01, connStr=0.5, convDT=0.05, convTau=0.5)

In [ ]:
axsim.crosstalk_test_three_node_positive(mc, ['MultivariateTE'], testConv=True, nCTDiscr=10, nData=2000,
                  nTest=50, maxLag=5, pTHR=0.01, noiseFrac=0.01, connStr=0.5, convDT=0.05, convTau=0.5)

In [ ]:
nTest = 20
maxLag = 5

idtxlParam = {
    'dim_order'       : 'ps',
    'cmi_estimator'   : 'JidtGaussianCMI',
    'max_lag_sources' : maxLag,
    'min_lag_sources' : 0,
    'parallelTrg' : False
}

for method in ['BivariateTE', 'MultivariateTE']:
    print('Doing method', method)
    rezMat = np.zeros((maxLag+1, maxLag+1))
    for lagTrue in range(0, maxLag+1):
        for iTest in range(nTest):
            data = two_node_system(200, [lagTrue], [0.5], noiseFrac=0.01, convDT=0.05, convTau=0.5, crossYX=0.2)
            mc.set_data(data, 'ps')
            rez = mc.metric3D(method, '', metricSettings=idtxlParam)            
            if not np.isnan(rez[1, 0, 1]):
                lagEst = int(rez[1, 0, 1])
                rezMat[lagTrue][lagEst] += 1
    
    print("Accuracy", acc(rezMat, nTest))
    print("Accuracy-corrected", acc_te_lag_corrected(rezMat, nTest))
    
    plt.figure()
    plt.imshow(rezMat)
    plt.show()